In [1]:
%matplotlib inline
import random
from time import sleep

import gym
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt


import gym
from gym.wrappers import Monitor
import gym_ple


couldn't import doomish
Couldn't import doom


In [2]:
class Network:
    
    def __init__(self):
        _fc_size = [256, 128, 2]
        self.fully_layers = list()
        n_fc_size = len(_fc_size)
        
        self.imh, self.imw, self.imc = (512, 288, 3)
        self.input_state = tf.placeholder(tf.float32, shape=[None, self.imh, self.imw, self.imc])
        self.gimg = tf.image.rgb_to_grayscale(self.input_state)
        self.rimg = tf.image.resize_images(self.gimg, [80, 80])
        
        self.conv1 = tf.layers.conv2d(self.rimg, 32, 8, activation=tf.nn.relu, strides=(1, 1),)
        self.max1 = tf.layers.max_pooling2d(self.conv1, 2, 2)

        self.conv2 = tf.layers.conv2d(self.max1, 64, 4, activation=tf.nn.relu)
        self.max2 = tf.layers.max_pooling2d(self.conv2, 2, 2)
        
        self.conv3 = tf.layers.conv2d(self.max2, 64, 3, activation=tf.nn.relu)
        self.max3 = tf.layers.max_pooling2d(self.conv3, 2, 2)
#         self.conv3 = tf.layers.conv2d(self.max2, 4, 5, activation=tf.nn.relu)
#         self.max3 = tf.layers.max_pooling2d(self.conv3, 2, 2)

        self.flattern = tf.contrib.layers.flatten(self.max2)
        for i in range(n_fc_size):
            if i == 0:
                self.fully_layers.append( self._create_fc(self.flattern, _fc_size[0]) )
            else:
                self.fully_layers.append( self._create_fc(self.fully_layers[-1], _fc_size[i]) )
        self.pred_Q = self.fully_layers[-1]
        self.select_action = tf.argmax(self.pred_Q, 1)
        self.expect_Q = tf.placeholder(shape=[None, 2], dtype=tf.float32)
        self.loss = tf.losses.mean_squared_error(self.expect_Q, self.pred_Q)
        self.trainer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(
            loss=self.loss, global_step=tf.train.get_global_step()
        )
    
    def _create_fc(self, input_layer, out_size):
        return tf.contrib.layers.fully_connected(input_layer, out_size, activation_fn=tf.nn.softmax)
    
    def predict(self, sess, state):
        select_action, pred_Q = sess.run([self.select_action, self.pred_Q], \
                                         feed_dict={self.input_state:state})
        return select_action, pred_Q
    
    def update(self, sess, X, Y):
        loss, _ = sess.run([self.loss, self.trainer], feed_dict={self.input_state:X, self.expect_Q:Y})
        return loss
    
class Agent:
    def __init__(self, session, beta=0.9, discount=0.3, batch_iter=10, batch_size=16, max_memory=1000):
        self.sess = session
        self.net = Network()
        self.beta = beta
        self.experience_memory = list()
        self.max_memory = max_memory
        self.batch_size = batch_size
        self.batch_iter = batch_iter
        self.discount = discount
        
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        self.sess.close()
    
    def initvar(self):
        self.sess.run(tf.global_variables_initializer())
        
    def memory(self, pack):
        if len(self.experience_memory) > self.max_memory:
            self.experience_memory.pop(0)
        self.experience_memory.append( pack )
    
    def predict(self, state):
        is_random = False
        select_action, pred_Q = self.net.predict(self.sess, state)
        if np.random.rand(1) < self.beta:
            is_random = True
            select_action = [env.action_space.sample()]
        return select_action, pred_Q, is_random
    
    
    def replay_experience(self):
        losses = list()
        for batchiter in range(self.batch_iter):
            X = list()
            Y = list()
            batch = random.sample(self.experience_memory, \
                min(len(self.experience_memory), self.batch_size))

            for state, action, reward, next_state, done in batch:
                pq = self.predict([state])[1]
                if done:
                    pq[0, action] = reward
                else:
                    pq[0, action] = reward + self.discount * np.max(self.predict([next_state])[1])
                X.append(state)
                Y.append(pq[0])
            loss = self.net.update(sess, np.array(X), np.array(Y))
            losses.append( loss )
        return np.mean(losses)



In [3]:
net = Network()
print(net.max2.shape)
# print(net.gimg.reshape([1, 80, 80, 1]))

(?, 16, 16, 64)


In [ ]:
env = gym.make('FlappyBird-v0')
state = env.reset()
print(state.shape)

(512, 288, 3)


In [ ]:
nepisode = 1000
steps = []
memory = []

with tf.Session() as sess:
    agent = Agent(sess)    
    sess.run(tf.global_variables_initializer())
    
    for episode in range(nepisode+1):
        done = False
        step = 0
        state = env.reset()
        while not done:
            print(np.sum(state), end=' ')
            action, pred_Q, _ = agent.predict([state])
            next_state, reward, done, _ = env.step(action[0])
            
            flat_state = sess.run([agent.net.flattern], feed_dict={agent.net.input_state:[state]})
            print(np.sum(flat_state))
            # Big Penalty to Agent
            if done: 
                reward = -100
            print(pred_Q, reward, end='')
            
            if reward == 0 and np.random.rand(1) < 0.95:
                    pass
            else:
                agent.memory( (state, action, reward, next_state, done) )
                print(' -mem-',end='')
            step += 1
            env.render()
            state = next_state
            print()
        print()
        print()
        steps.append(step)
        agent.beta = agent.beta*0.7    
        if episode%10 == 0 and episode != 0:
            loss = agent.replay_experience()
            print("in Episode {:4d} mean step: {:.2f}, Agent Replay Experience, get loss: {:.2f} "\
                      .format(episode, np.mean(steps[episode-10:episode+1]), loss))

    
    step = 0
    done = False
    state = env.reset()
    while not done:
        action, Q, _ = agent.predict([state])
        state, reward, done, _ = env.step(action[0])
        step += 1
        env.render()
        sleep(0.1)
    print('Bot play game and archive {} step'.format(step))
    env.close()
    

32130 80.8301
[[ 0.50053793  0.49946204]] 0.0
54145472 80610.8
[[ 0.50061107  0.49938893]] 0.0
54145472 80418.1
[[ 0.50061107  0.49938899]] 0.0
54145472 80470.5
[[ 0.50061107  0.49938893]] 0.0
54145472 80668.0
[[ 0.50061113  0.49938896]] 0.0
54145472 80600.9
[[ 0.50061107  0.49938893]] 0.0
54145472 80589.4
[[ 0.50061107  0.49938893]] 0.0
54135392 80297.7
[[ 0.50061113  0.49938896]] 0.0
54105904 80325.4
[[ 0.50061113  0.49938896]] 0.0
54195642 80803.2
[[ 0.50061113  0.49938896]] 0.0
54353796 80768.9
[[ 0.50061107  0.49938893]] 0.0
54580944 80558.6
[[ 0.50061107  0.49938893]] 0.0
54839556 80824.4
[[ 0.50061107  0.49938896]] 0.0
55075824 80741.0
[[ 0.50061107  0.49938893]] 0.0
55265060 80602.0
[[ 0.50061113  0.49938896]] 0.0
55411144 81113.7
[[ 0.50061113  0.49938896]] 0.0
55490558 82164.2
[[ 0.50061113  0.49938896]] 0.0
55513290 83372.1
[[ 0.50061113  0.49938896]] 0.0
55504480 84237.7
[[ 0.50061113  0.49938896]] 0.0
55446196 83768.5
[[ 0.50061113  0.49938896]] 0.0
55439504 85140.8
[[ 0.5

55207332 86459.3
[[ 0.50061113  0.49938896]] 0.0
55211204 86745.7
[[ 0.50061113  0.49938896]] 0.0
55212172 86586.5
[[ 0.50061113  0.49938896]] 0.0
55204480 86668.3
[[ 0.50061113  0.49938896]] 0.0
55204048 86595.4
[[ 0.50061113  0.49938896]] 0.0
55204480 86763.6
[[ 0.50061113  0.49938896]] 0.0
55191184 87259.1
[[ 0.50061113  0.49938896]] 0.0
55170740 86814.7
[[ 0.50061113  0.49938896]] 0.0
55279068 86486.8
[[ 0.50061113  0.49938896]] 0.0
55439392 86275.8
[[ 0.50061113  0.49938896]] 0.0
55658728 85771.5
[[ 0.50061107  0.49938893]] 0.0
55898120 85805.8
[[ 0.50061107  0.49938893]] 0.0
56102632 85530.3
[[ 0.50061107  0.49938893]] 0.0
56249860 85829.3
[[ 0.50061113  0.49938896]] 0.0
56354644 86285.3
[[ 0.50061113  0.49938896]] 0.0
56407024 87233.9
[[ 0.50061113  0.49938896]] 0.0
56416288 87138.0
[[ 0.50061113  0.49938896]] 0.0
56407416 87228.0
[[ 0.50061113  0.49938896]] 0.0
56363300 87258.2
[[ 0.50061113  0.49938896]] 0.0
56361120 87187.0
[[ 0.50061113  0.49938896]] 0.0
56352542 87186.1
[[ 

55208572 87457.7
[[ 0.5006085   0.49939144]] 0.0
55209488 87339.3
[[ 0.50061035  0.49938956]] 0.0
55210404 87712.2
[[ 0.50061101  0.49938908]] 0.0
55212772 87654.4
[[ 0.50061107  0.49938902]] 0.0
55213740 87691.9
[[ 0.50061107  0.49938896]] 0.0
55206532 87790.6
[[ 0.50061107  0.49938893]] 0.0
55206584 87585.2
[[ 0.50061107  0.49938893]] 0.0
55207016 87665.8
[[ 0.50061107  0.49938893]] 0.0
55193720 88229.1
[[ 0.50061107  0.49938893]] 0.0
55144424 88347.1
[[ 0.50061107  0.49938896]] 0.0
55169912 88220.4
[[ 0.50061107  0.49938893]] 0.0
55219280 87955.0
[[ 0.50061107  0.49938893]] 0.0
55322482 87481.2
[[ 0.50061113  0.4993889 ]] 0.0
55467598 87330.5
[[ 0.50061107  0.49938899]] 0.0
55637386 87625.6
[[ 0.50061107  0.49938893]] 0.0
55825330 87620.1
[[ 0.50061101  0.49938899]] 0.0 -mem-
56025778 87816.4
[[ 0.50061107  0.49938896]] 0.0
56194320 88410.8
[[ 0.50061113  0.49938896]] 0.0
56314476 88140.3
[[ 0.50061107  0.49938893]] 0.0
56386478 88082.2
[[ 0.50061107  0.49938893]] 0.0
56366320 88204

55537032 83682.4
[[ 0.50061113  0.49938896]] 0.0
55500616 84635.0
[[ 0.50061113  0.49938896]] 0.0
55433172 84332.1
[[ 0.50061113  0.49938896]] 0.0
55426480 84804.2
[[ 0.50061113  0.49938896]] 0.0
55426160 85552.3
[[ 0.50061113  0.49938896]] 0.0
55422720 86339.9
[[ 0.50061113  0.49938896]] 0.0 -mem-
55411052 87859.7
[[ 0.50061113  0.49938896]] 0.0 -mem-
55410136 88563.8
[[ 0.50061113  0.49938896]] 0.0
55408520 89119.0
[[ 0.50061113  0.49938896]] 0.0
55412168 89426.5
[[ 0.50061113  0.49938896]] 0.0
55409264 89429.0
[[ 0.50061113  0.49938896]] 0.0
55409696 89383.2
[[ 0.50061113  0.49938896]] 0.0
55405964 89385.2
[[ 0.50061113  0.49938896]] 0.0
55399136 89317.1
[[ 0.50061113  0.49938896]] 0.0
55391340 89381.4
[[ 0.50061113  0.49938896]] 0.0
55386016 89772.8
[[ 0.50061113  0.49938896]] 0.0 -mem-
55380260 90326.9
[[ 0.50061113  0.49938896]] 0.0
55380796 91210.7
[[ 0.50061113  0.49938896]] 0.0
55385420 91062.7
[[ 0.50061113  0.49938896]] 0.0
55384832 90986.8
[[ 0.50061113  0.49938896]] 0.0
55

77369138 126512.0
[[ 0.50061113  0.49938896]] 0.0
77366700 126529.0
[[ 0.50061107  0.49938899]] 0.0 -mem-
77365360 125094.0
[[ 0.50061107  0.49938893]] 0.0
77390530 124338.0
[[ 0.50061113  0.49938896]] 0.0
77415516 126683.0
[[ 0.50061113  0.49938896]] -100 -mem-


in Episode   10 mean step: 61.18, Agent Replay Experience, get loss: 1136.25 
77440794 234509.0
[[ 0.49018964  0.50981039]] 0.0
82508884 240239.0
[[ 0.49018964  0.50981039]] 0.0
82508884 240318.0
[[ 0.49018964  0.50981039]] 0.0
82508884 240453.0
[[ 0.49018964  0.50981039]] 0.0
82508884 240578.0
[[ 0.49018964  0.50981039]] 0.0
82508884 240474.0
[[ 0.49018964  0.50981039]] 0.0
82508884 240319.0
[[ 0.49018964  0.50981039]] 0.0
82476628 240224.0
[[ 0.49018964  0.50981039]] 0.0
82137536 240353.0
[[ 0.49018964  0.50981039]] 0.0
81900412 240095.0
[[ 0.49018964  0.50981039]] 0.0
81718868 240461.0
[[ 0.49018964  0.50981039]] 0.0
81603004 240488.0
[[ 0.49018964  0.50981039]] 0.0
81525184 240615.0
[[ 0.49018964  0.50981039]] 0.0
8144695

...

80012116 347059.0
[[ 0.5206393  0.4793607]] 0.0
79657172 346794.0
[[ 0.5206393  0.4793607]] 0.0
79385272 346908.0
[[ 0.5206393  0.4793607]] 0.0
79146964 347845.0
[[ 0.5206393  0.4793607]] 0.0
78974080 347996.0
[[ 0.5206393  0.4793607]] 0.0
78855286 347810.0
[[ 0.5206393  0.4793607]] 0.0
78769548 347856.0
[[ 0.5206393  0.4793607]] 0.0
78706370 347308.0
[[ 0.5206393  0.4793607]] 0.0
78649274 346711.0
[[ 0.5206393  0.4793607]] 0.0
78546608 347495.0
[[ 0.5206393  0.4793607]] 0.0
78375428 346903.0
[[ 0.5206393  0.4793607]] 0.0
78139574 346998.0
[[ 0.5206393  0.4793607]] 0.0
77808718 347748.0
[[ 0.5206393  0.4793607]] 0.0
77774922 347963.0
[[ 0.5206393  0.4793607]] 0.0
77752466 347031.0
[[ 0.5206393  0.4793607]] 0.0
77750808 347189.0
[[ 0.5206393  0.4793607]] 0.0 -mem-
77755460 345686.0
[[ 0.5206393  0.4793607]] 0.0
77782344 344271.0
[[ 0.5206393  0.4793607]] 0.0
77808495 344926.0
[[ 0.5206393  0.4793607]] -100 -mem-


77833735 345182.0
[[ 0.5206393  0.4793607]] 0.0
54145472 221956.0
[[ 0.52

55386092 239646.0
[[ 0.5206393  0.4793607]] 0.0
55379264 239970.0
[[ 0.5206393  0.4793607]] 0.0
55372972 239900.0
[[ 0.5206393  0.4793607]] 0.0
55367648 240146.0
[[ 0.5206393  0.4793607]] 0.0
55361892 240585.0
[[ 0.5206393  0.4793607]] 0.0
55362428 240921.0
[[ 0.5206393  0.4793607]] 0.0
55367052 240996.0
[[ 0.5206393  0.4793607]] 0.0
55367968 240864.0
[[ 0.5206393  0.4793607]] 0.0
55368884 240596.0
[[ 0.5206393  0.4793607]] 0.0
55371252 241127.0
[[ 0.5206393  0.4793607]] 0.0
55372972 241054.0
[[ 0.5206393  0.4793607]] 0.0
55365764 240887.0
[[ 0.5206393  0.4793607]] 0.0
55365816 241411.0
[[ 0.5206393  0.4793607]] 0.0
55366248 241498.0
[[ 0.5206393  0.4793607]] 0.0
55352952 241669.0
[[ 0.5206393  0.4793607]] 0.0
55304924 241475.0
[[ 0.5206393  0.4793607]] 0.0
55326364 241572.0
[[ 0.5206393  0.4793607]] 0.0
55382164 241679.0
[[ 0.5206393  0.4793607]] 0.0
55503904 241623.0
[[ 0.5206393  0.4793607]] 0.0
55677888 241676.0
[[ 0.5206393  0.4793607]] 0.0
55884084 241772.0
[[ 0.5206393  0.479360

82495096 349154.0
[[ 0.5234068   0.47659329]] 0.0
82173518 349223.0
[[ 0.5234068   0.47659329]] 0.0
81970624 349265.0
[[ 0.5234068   0.47659329]] 0.0
81823474 349284.0
[[ 0.5234068   0.47659329]] 0.0
81731916 349389.0
[[ 0.5234068   0.47659329]] 0.0
81660316 349447.0
[[ 0.5234068   0.47659329]] 0.0
81557992 349371.0
[[ 0.5234068   0.47659329]] 0.0
81406874 349029.0
[[ 0.5234068   0.47659329]] 0.0
81217378 349557.0
[[ 0.5234068   0.47659329]] 0.0
80972192 349741.0
[[ 0.5234068   0.47659329]] 0.0
80678144 349298.0
[[ 0.5234068   0.47659329]] 0.0
80364112 349292.0
[[ 0.5234068   0.47659329]] 0.0
80007960 348701.0
[[ 0.5234068   0.47659329]] 0.0
79976772 348597.0
[[ 0.5234068   0.47659329]] 0.0
79966632 347981.0
[[ 0.5234068   0.47659329]] 0.0
79952440 347347.0
[[ 0.5234068   0.47659329]] 0.0
79937584 346352.0
[[ 0.5234068   0.47659329]] 0.0
79937200 345086.0
[[ 0.5234068   0.47659329]] 0.0
79936440 344576.0
[[ 0.5234068   0.47659329]] 0.0
79933916 343988.0
[[ 0.5234068   0.47659329]] 0.0


55561830 233844.0
[[ 0.5234068   0.47659329]] 0.0
55775724 233740.0
[[ 0.5234068   0.47659329]] 0.0
55977308 233805.0
[[ 0.5234068   0.47659329]] 0.0
56142492 233858.0
[[ 0.5234068   0.47659329]] 0.0
56272716 233501.0
[[ 0.5234068   0.47659329]] 0.0 -mem-
56356464 234362.0
[[ 0.5234068   0.47659329]] 0.0
56395638 234810.0
[[ 0.5234068   0.47659329]] 0.0
56408628 235277.0
[[ 0.5234068   0.47659329]] 0.0
56370322 235645.0
[[ 0.5234068   0.47659329]] 0.0
56368142 236711.0
[[ 0.5234068   0.47659329]] 0.0
56361540 237582.0
[[ 0.5234068   0.47659329]] 0.0
56358744 238460.0
[[ 0.5234068   0.47659329]] 0.0
56343142 238680.0
[[ 0.5234068   0.47659329]] 0.0
56344248 239428.0
[[ 0.5234068   0.47659329]] 0.0
56341070 240112.0
[[ 0.5234068   0.47659329]] -100 -mem-


56342568 240495.0
[[ 0.5234068   0.47659329]] 0.0
82527352 348991.0
[[ 0.5234068   0.47659329]] 0.0
82527352 348912.0
[[ 0.5234068   0.47659329]] 0.0
82527352 349119.0
[[ 0.5234068   0.47659329]] 0.0
82527352 349188.0
[[ 0.5234068   0.

55414598 236176.0
[[ 0.5234068   0.47659329]] 0.0
55412230 237133.0
[[ 0.5234068   0.47659329]] 0.0
55409326 237359.0
[[ 0.5234068   0.47659329]] 0.0
55409758 238210.0
[[ 0.5234068   0.47659329]] 0.0
55406026 239593.0
[[ 0.5234068   0.47659329]] 0.0 -mem-
55399198 240038.0
[[ 0.5234068   0.47659329]] 0.0
55392906 239808.0
[[ 0.5234068   0.47659329]] 0.0
55387582 239818.0
[[ 0.5234068   0.47659329]] 0.0
55381826 240203.0
[[ 0.5234068   0.47659329]] 0.0
55382362 240494.0
[[ 0.5234068   0.47659329]] 0.0
55386986 240731.0
[[ 0.5234068   0.47659329]] 0.0
55387902 240792.0
[[ 0.5234068   0.47659329]] 0.0
55388818 240592.0
[[ 0.5234068   0.47659329]] 0.0
55391186 240950.0
[[ 0.5234068   0.47659329]] 0.0
55392906 240771.0
[[ 0.5234068   0.47659329]] 0.0
55385698 240643.0
[[ 0.5234068   0.47659329]] 0.0
55385750 241026.0
[[ 0.5234068   0.47659329]] 0.0
55386182 241118.0
[[ 0.5234068   0.47659329]] 0.0
55372886 241507.0
[[ 0.5234068   0.47659329]] 0.0
55320054 241008.0
[[ 0.5234068   0.47659329]

In [ ]:
state = env.reset()

In [ ]:
state